In [2]:
import pandas as pd
import json
import ast
from tqdm import tqdm

In [ ]:
df = pd.read_csv('setlists.csv')

In [ ]:
def open_sets(row):
    try:
        sets = ast.literal_eval(row)
        music_list = [x['name'] for x in sets['set'][0]['song']]
        return music_list
    except:
        return 'no_info'

In [ ]:
#aplica a função em uma coluna   
df['music_name'] = df['sets'].apply(open_sets)

In [ ]:
artist_name = []

for x in df['artist']:
    name_dictionary = ast.literal_eval(x)
    artist_name.append(name_dictionary['name'])

In [ ]:
tour_name = []

for x in df['tour']:
    try:
        tour_dictionary = ast.literal_eval(x)
        tour_name.append(tour_dictionary['name'])
    except:
        pass

In [ ]:
venue_name = []
city_name = []
longitude_list = []
latitude_list = []
country_real = []

for x in df['venue']:
    venue_dictionary = ast.literal_eval(x)
    venue_name.append(venue_dictionary['name'])
    city_name.append(venue_dictionary['city']['name'])
    longitude_list.append(venue_dictionary['city']['coords']['long'])
    latitude_list.append(venue_dictionary['city']['coords']['lat'])
    country_real.append(venue_dictionary['city']['country']['name'])

In [ ]:
df["tour_name"] = pd.Series(tour_name)
df['artist_name'] = pd.Series(artist_name)
df['venue'] = pd.Series(venue_name)
df['city'] = pd.Series(city_name)
df['longitude'] = pd.Series(longitude_list)
df['latitude'] = pd.Series(latitude_list)
df['country'] = pd.Series(country_real)

In [ ]:
df_exploded = df.explode('music_name')

In [ ]:
df_exploded.drop(['versionId','lastUpdated','artist','tour','sets','url','info'], axis=1, inplace = True)

In [ ]:
df_exploded.to_csv('music_data_without_spotify.csv', index=False)

### Creating lists with artists names

In [ ]:
artists_name = df_exploded[['artist_name']]

In [ ]:
artists_name

In [ ]:
artists_name.drop_duplicates(inplace=True)

In [ ]:
artists_name.reset_index(inplace=True)

In [ ]:
artists_name.drop(axis=1,columns='index',inplace=True)

In [ ]:
artists_name_list = list(artists_name['artist_name'])

In [ ]:
spotify_client_id = open('client_id_spotify.txt', 'r').read()
spotify_client_secret = open('client_secret_spotify.txt', 'r').read()

In [ ]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id=spotify_client_id,client_secret=spotify_client_secret))

In [ ]:
artists_info = {}

for artist in tqdm(artists_name_list):
        artists_info[artist] = sp.search(q=artist,type='artist',limit=1)

In [ ]:
spotify_artists_data = pd.DataFrame()
for singer in tqdm(artists_info.keys()):
    df_artist_spotify = pd.DataFrame(artists_info[singer]['artists']['items'])
    df['artist'] = singer
    spotify_artists_data = spotify_artists_data.append(df_artist_spotify)

In [ ]:
spotify_artists_data.reset_index(inplace=True)
spotify_artists_data.drop('index',axis=1,inplace=True)

In [ ]:
followers = []
for x in spotify_artists_data['followers']:
    follower_dict = dict(spotify_artists_data['followers'])
    for n in follower_dict.keys():
        followers.append(follower_dict[n]['total'])

In [ ]:
spotify_artists_data["qty_followers"] = pd.Series(followers)
spotify_artists_data.drop('followers',axis=1,inplace=True)

In [ ]:
spotify_artists_data.drop(['external_urls','genres','href','images','type','uri'],axis=1,inplace=True)

In [ ]:
spotify_artists_data.to_csv('artists_spotify.csv',index=False)

In [ ]:
id_list = list(spotify_artists_data['id'])

In [ ]:
albums_info = {}

for id in tqdm(id_list):
        albums_info[id] = sp.artist_albums(id)

In [ ]:
spotify_albums_data = pd.DataFrame()
for id in tqdm(albums_info.keys()):
    df_albums_spotify = pd.DataFrame(albums_info[id]['items'])
    spotify_albums_data = spotify_albums_data.append(df_albums_spotify)

In [ ]:
spotify_albums_data.drop(['album_group','available_markets','external_urls','href','images','uri'], axis=1 ,inplace=True)

In [ ]:
spotify_albums_data.reset_index(inplace=True)

In [ ]:
spotify_albums_data.drop('index', axis=1 ,inplace=True)

In [ ]:
artist_id = []
artist_name = []

for x in tqdm(range(len(spotify_albums_data['artists']))):
    artist_name.append(spotify_albums_data['artists'][x][0]['name'])
    artist_id.append(spotify_albums_data['artists'][x][0]['id'])

In [ ]:
spotify_albums_data["artist_name"] = pd.Series(artist_name)
spotify_albums_data['artist_id'] = pd.Series(artist_id)

In [ ]:
spotify_albums_data.drop(['album_type','artists'],axis=1,inplace=True)

In [ ]:
spotify_albums_data.to_csv('spotify_albums_data.csv')

In [3]:
spotify_albums_data = pd.read_csv('spotify_albums_data.csv')

In [4]:
albums_id = list(spotify_albums_data['id'])

In [5]:
albums_id = spotify_albums_data['id']

In [ ]:
albums_id_1 = albums_id[:20001]
albums_id_2 = albums_id[20001:40001]
albums_id_3 = albums_id[40001:]


In [ ]:
tracks_info_1 = {}

for id in tqdm(albums_id_1):
        tracks_info_1[id] = sp.album_tracks(id)

In [ ]:
spotify_tracks_data_1 = pd.DataFrame()
for id in tqdm(tracks_info.keys()):
    df_tracks_spotify = pd.DataFrame(tracks_info[id]['items'])
    spotify_tracks_data_1 = spotify_tracks_data.append(df_albums_spotify)